# ✔	 - Import modules -

In [1]:
import os, time, random, json
import numpy as np
import scipy.stats as stats

In [2]:
from math import exp, cos
from tqdm import tqdm

In [3]:
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20)
plt.rcParams["axes.labelweight"] = "bold"

# ✔	 - Function definition -

# ✘ Corrected Input function

In [4]:
## Function returning the corrected Input (x = 0 if x <= 0  or  x = x if x >= 0)

def Ic(Input=0):
    
    if Input <= 0:
        return 0
    
    else:
        return (Input)

# ✘ Activity at time t-Delta (delay) function

In [5]:
## function to get the index "-delay/dt" in the list representing m(t-Delta)

def mdelta(synapse_value, delay, t, dt,i):
    
    if t < delay:  # if the delay is the time is too small, we take m(t-delta)=0
        return 0
    
    else:  # if the time is above or equal to the delay, we can find the m(t-delta)
        return synapse_value[int(-delay / dt)][i]

# ✘ Plotting functions (Inputs & activities) and data folder creation function

In [6]:
## plot the external Inputs

def Inplot(x,data,title):    
    
    ind=np.arange(len(data))
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.barh(ind, data, 0.899,color='g')
    ax.set_yticks(ind)
    r = ax.set_yticklabels(x, ha = 'left',fontsize = 16)
    fig.set_size_inches(15, 2, forward=True)
    
    plt.draw()
    yax = ax.get_yaxis()
    pad = max(T.label.get_window_extent().width for T in yax.majorTicks)
    yax.set_tick_params(pad=pad,)
    
    ######################################## Annotations on plot #############################################
    bbox_props = dict(boxstyle="round4,pad=0.3", fc="white", ec="k", lw=2)
    t = ax.text(H['Gpe'], 1, str(H['Gpe']), ha="center", va="center", rotation=5,
            size=12,
            bbox=bbox_props) 
    bbox_props = dict(boxstyle="round4,pad=0.3", fc="white", ec="k", lw=2)
    t = ax.text(H['Fsi'], 2, str(H['Fsi']), ha="center", va="center",rotation=5,
            size=12,
            bbox=bbox_props) 
    bbox_props = dict(boxstyle="round4,pad=0.3", fc="white", ec="k", lw=2)
    t = ax.text(H['Msn'], 0, str(H['Msn']), ha="center", va="center",rotation=5,
            size=12,
            bbox=bbox_props) 
    ##########################################################################################################
    ax=plt.gca()
    col_labels=['Fsi','Msn','Gpe']
    row_labels=['G (Weight)','Basal Activity (spk/s)']          
    table_vals=[[str(G['MsnFsi']),str(G['GpeMsn']),str(G['FsiGpe'])],
                [str(M['MsnFsi']),str(M['GpeMsn']),str(M['FsiGpe'])]]
    
    table=plt.table(cellText=table_vals,
        colWidths = [0.4]*3,
        rowLabels=row_labels,
        colLabels=col_labels,
        loc='bottom',bbox=[0.4, -0.65, 0.2, 0.3])
    table.set_fontsize(10)

    plt.draw()
    plt.xlabel('spike per second (spk/s)')
    plt.ylabel('structures')
    plt.title('External Input value for each structure')

In [7]:
## plot activity and input dynamics

def plotting(k,typ, title, xax, yax, element, source,value, v1, v2, v3,On): 
    
    fig = plt.figure(figsize=(15,10))
    ax=fig.add_subplot(111) 
    
    for element in source:
        value[element]=np.zeros(n_period)
        
        for t in range(0,n_period):
            if On == 0:
                value[element][t] = typ[element][t][k]
            else:
                value[element][t] = np.mean(typ[element][t])     

        ax.plot(time_value*1000, value[element], label= str(element),linewidth=2)   
        ax.set_title((title + ' {}'.format(k+1)))
        ax.set_ylabel(yax), ax.set_xlabel(xax)
        ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0,prop={'size':15})
        
    
    ax=plt.gca()
    col_labels=['Fsi','Msn','Gpe']
    row_labels=['G (Weight)','Min','Mean','Max','Var','Stand.Dev']          
    table_vals=[[str(G['MsnFsi']),str(G['GpeMsn']),str(G['FsiGpe'])],
                [np.amin(value[v1]),np.amin(value[v2]),np.amin(value[v3])],
                [np.mean(value[v1]),np.mean(value[v2]),np.mean(value[v3])],
                [np.amax(value[v1]),np.amax(value[v2]),np.amax(value[v3])],
                [np.var(value[v1]),np.var(value[v2]),np.var(value[v3])],
                [np.std(value[v1]),np.std(value[v2]),np.std(value[v3])]]
    
    table=plt.table(cellText=table_vals,
        colWidths = [0.8]*5,
        rowLabels=row_labels,
        colLabels=col_labels,
        loc='bottom',bbox=[0.27, -0.4, 0.6, 0.3])
    table.set_fontsize(20)

In [8]:
## will create the files needed to store values in CWD/Data

def data_dir(dir="os.getcwd()"):            
    if not os.path.exists('{}/{}'.format(os.getcwd(),'DataFsi')): # will be used to tag simulation results
        directory = os.path.dirname(dir)
        os.makedirs('{}/{}'.format(os.getcwd(),'DataFsi'))
    if not os.path.exists('DataFsi/{}'.format(timestr)):
        os.makedirs('DataFsi/{}'.format(timestr))

In [9]:
## Plot the spectrum and return the plot of 3 populations

def Spectrum(): 
    col=['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    FourNo = dict()
    fourier= dict()
    freq=dict()
#=========================================================================================

    fig, axes = plt.subplots(1, 3, figsize=(30, 9))

    i=0
    for con in connections:
        FourNo[con]=Mean[con] - np.mean(Mean[con])
#=========================================================================================

#Calculating FFT of signal
        fourier[con]=np.fft.fft(FourNo[con])/len(FourNo[con])
        freq[con]=np.fft.fftfreq(fourier[con].shape[0],dt)

        toc=np.argmax(abs(fourier[con]))
    
#Plotting FFT of signal
        axes[i].plot(freq[con],abs(fourier[con]),label=str(abs(freq[con][toc]))+' Hz',
                 color=col[i+2],linewidth=3)
        axes[i].set_title(con,fontsize=20)
        axes[i].set_xlabel('spectrum range (Hz)', fontsize=18)
        axes[i].set_xlim(0,50)
        axes[i].set_ylabel('Amplitude (normalized)',fontsize=18)
        axes[i].legend(bbox_to_anchor=(0.95, 0.95), loc=1, borderaxespad=0. , fontsize=20)
        i+=1
            
    plt.savefig('DataFsi/{}/FFT_{}72.png'.format(timestr,'Mean'),dpi=72,bbox_inches='tight')
    plt.savefig('DataFsi/{}/FFT_{}150.png'.format(timestr,'Mean'),dpi=150,bbox_inches='tight')
    plt.savefig('DataFsi/{}/FFT_{}300.png'.format(timestr,'Mean'),dpi=300,bbox_inches='tight')
    plt.show()

# ✔	 - Initial parameters - 

In [10]:
## Initialization of the random generator (reproductibility)

w = np.random.randint(1,10)

np.random.seed(w)

In [11]:
## Population size (number of neuron)
n = 1

## Default Time beginning
t = 0                                       # (sec)

## Default trial duration
duration = 1.00                             # (sec)

## Default Time resolution
dt = 0.0001                                 # 0.00001 (sec)

## Period occuring 
n_period = int(duration / dt) + 1

In [12]:
## create the time array
time_value = np.arange(0, duration, dt)     

In [13]:
## creation of dict to store m, dm, J, H and input I values
Mean, m, dm, J, H, y,value,I = dict(),dict(), dict(), dict(), dict(), dict(), dict(), dict()

In [14]:
connections = ["GpeMsn", "FsiGpe", "MsnFsi"]
structures = ["Msn", "Fsi", "Gpe"]

In [15]:
 ## Fixed point activity:
    
M = {                     
    "GpeMsn" :       5,                             # spikes/s
    "MsnFsi" :       5,                             # spikes/s
    "FsiGpe" :       20                             # spikes/s
}

In [16]:
 ## Weights (connexion pre -> post ; notation: PostPre):
    
G = {
    "GpeMsn" :       3.00,
    "FsiGpe" :       0.50,
    "MsnFsi" :       2.00,     
}

In [17]:
 ## Tau:
    
tau = {
    "GpeMsn" :       0.020,                         # time decay constant MSN to GPe : 0.020 (sec)
    "FsiGpe" :       0.005,                         # time decay constant GPe to FSi : 0.005 (sec)
    "MsnFsi" :       0.005,                         # time decay constant FSi to MSN : 0.005 (sec)
}

In [18]:
 ## Delays:
    
D = {
    "GpeMsn" :       0.0070,                        # 7ms: Jaeger & Kita, 2011 (in vivo-rat & monkey)
    "FsiGpe" :       0.0010,                        # 1ms: Jaeger & Kita, 2011 (in vivo-rat & monkey)
    "MsnFsi" :       0.0008,                        # 0.8ms: Gittis et al., 2010 (in vitro - mice)
}

In [19]:
 ## Connection probability:
    
C = {
    "GpeMsn" :       1.00,                          # all-to-all
    "FsiGpe" :       1.00,
    "MsnFsi" :       1.00,                          # all-to-all
}

In [20]:
 ## External Input: calculated and constant during the whole simulation
    
H = {
    "Msn" :          M['GpeMsn'] + G['MsnFsi'] * M['MsnFsi'],   
    "Fsi" :          M['MsnFsi'] + G['FsiGpe'] * M['FsiGpe'],   
    "Gpe" :          M['FsiGpe'] + G['GpeMsn'] * M['GpeMsn']
}

In [21]:
 ## Noise (sigma): set the noise from a gaussian distribution
    
N = {
    "Msn":H['Msn']/5,                        # set to '0' to eliminate the noise
    "Fsi":H['Fsi']/5,   
    "Gpe":H['Gpe']/5
}

In [22]:
input_mapping = {
    "Msn": "MsnFsi",
    "Gpe": "GpeMsn",
    "Fsi": "FsiGpe"
}

activities_mapping = {
    "GpeMsn": "Msn",
    "FsiGpe": "Gpe",
    "MsnFsi": "Fsi"
}

In [23]:
## create an array of 'nperiod' array (store value trought time) with n zeroes in

for con in connections:
    
    m[con] = np.zeros((n_period, n))
   
    y[con] = int(D[con]//dt)
    
    m[con][0:y[con]] = M[con] #    m[con][0:y[con]:n]?
    
## create an array composed of     
    dm[con] = np.zeros((n_period, n))
    
## create an array composed of 0 & 1 depending on the probability "Co"(max=1=100%)
    J[con] = np.random.choice(2, size=(n, n), p=[(1-C[con]),C[con]])

In [24]:
 ## Threshold distribution (sigma):
    
TN = {
    "Msn": H["Msn"]/1,   
    "Fsi": 1, #H["Fsi"]/40,   
    "Gpe": 1 #H["Gpe"]/40
}

In [25]:
T = dict()

In [26]:
## inputs array generation: noises picked (N=sigma)

for stc in structures:
    
    T[stc] = np.random.normal(0, TN[stc] , size=n)
    
    I[stc] = np.random.normal(0, N[stc], size=(n_period, n)) + H[stc] + T[stc]

# ✔	 - Simulation Core -

In [27]:
for t in tqdm(range(1, n_period)):    

## Update of Inputs ================================================================================ 
    
    for stc in structures:
        
        for i in range(n):
            
            con = input_mapping[stc]               # multiplication by the probability of connexion below with n
            
            I[stc][t][i] += - np.sum(
                G[con]* mdelta(m[con][:t + 1], D[con], time_value[t], dt,i) *J[con][i]
                )/(np.sum(J[con][i]))  

## Update of Activities ============================================================================ 

    for con in connections:          
        
        for i in range(n):
            
            stc = activities_mapping[con]
            
            dm[con][t][i] = dt * (- m[con][t][i] + Ic(I[stc][t][i])) / tau[con]
            
        if t < n_period-1:                             # we can't add last value of dm to the last value of m
            
            m[con][t+1]=m[con][t]+dm[con][t]
            
        else:
            continue
            

 96%|█████████▌| 9623/10000 [00:00<00:00, 23878.80it/s]


IndexError: index 10000 is out of bounds for axis 0 with size 10000

# ✔	 - Result Plots & Save data -

In [ ]:
timestr = time.strftime("%Y-%m-%d-%H:%M:%S") # give the date and time used to tag simulation results
data_dir()  # creating the 'Data' directory on CWD and the directory (with date & time) to store the results

for stc in structures:  # save the Input array (npy format) for each neuron population in "structures"
    np.save('DataFsi/{}/{}_Inputs'.format(timestr, stc), I[stc])
    
for con in connections: # save the Activity array (npy format) for each neuron population in "connections"
    np.save('DataFsi/{}/{}_Activities'.format(timestr, con), m[con])

datax={'Weight':G,'Delay':D,'tau':tau,'Noise':N,'BasalActivity':M,'ConnexProba':C,  # make one dict with all parameters used 
       'ExternalInput':H,'Neuron_number':n,'time_trial':t,'duration':duration,'step':dt}

with open('DataFsi/{}/0_Parameters.json'.format(timestr),'w') as f: # save the parameters used during the simulation
            json.dump(datax, f) 

In [ ]:
## Plotting EXTERNAL INPUTS
Inplot(structures,[H['Msn'],H['Gpe'],H['Fsi']],'External Input value for each structure')
plt.savefig('DataFsi/{}/0_H.png'.format(timestr),dpi=50,bbox_inches='tight')
plt.show()

In [ ]:
## Plotting INPUTS over time
for k in range(0,n):
    plotting(k,I,'Input over time: Neuron','Time (ms)','Input (spk/s)',stc,structures,value,'Fsi','Msn','Gpe',On=0)
    plt.savefig('DataFsi/{}/In{}.png'.format(timestr,k+1),dpi=300,bbox_inches='tight')
    plt.close()
    plt.show()

In [ ]:
## Plotting ACTIVITIES over time
for k in range(0,n):
    plotting(k,m,'Activity over time: Neuron','Time (ms)','Activity (spk/s)',con,connections,value,'MsnFsi','GpeMsn','FsiGpe',On=0)
    plt.savefig('DataFsi/{}/Mn{}.png'.format(timestr,k+1),dpi=300,bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
## Plotting one neuron randomly
k=np.random.randint(0,n)
plotting(k,m,'Activity ONE RANDOM Neuron','Time (ms)','Activity (spk/s)',con,connections,value,'MsnFsi','GpeMsn','FsiGpe',On=0)
plt.savefig('DataFsi/{}/Mn{}.png'.format(timestr,k+1),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))


ax1 = plt.subplot(311)
ax1.set_title('RANDOM NEURON',fontsize=30)
#ax1.set_ylim(0,0.5)
ax1.set_ylabel('MSN',fontsize=20,fontweight='bold')
plt.plot(time_value*1000, value['GpeMsn'],'g',linewidth=2)
plt.setp(ax1.get_xticklabels(), visible=False)

    # share x only
ax2 = plt.subplot(312, sharex=ax1)
ax2.set_ylabel('Gpe',fontsize=20,fontweight='bold')
plt.plot(time_value*1000, value['FsiGpe'],'r',linewidth=2)

# make these tick labels invisible
plt.setp(ax2.get_xticklabels(), visible=False)

# share x and y
ax3 = plt.subplot(313, sharex=ax1)
#ax3.set_ylim(2,5)
ax3.set_ylabel('FSI',fontsize=20,fontweight='bold')
ax3.set_xlabel('Time (ms)',fontsize=25,fontweight='bold')
plt.plot(time_value*1000, value['MsnFsi'],'b',linewidth=2)

fig.text(0.020, 0.5, "Activities (Spikes/s)", rotation="vertical", va="center",fontsize=20,fontweight='bold')
   
plt.savefig('DataFsi/{}/randnActivity72.png'.format(timestr),dpi=72,bbox_inches='tight')
plt.savefig('DataFsi/{}/randnActivity150.png'.format(timestr),dpi=150,bbox_inches='tight')
plt.savefig('DataFsi/{}/randnActivity300.png'.format(timestr),dpi=300,bbox_inches='tight')
plt.show()

# ✔	 - Mean firing rate plotting - 

In [ ]:
## Ploting the mean activity of the n neuron
plotting(n-1, m, 'Mean activities of all neuron(s) :  ', 'Time (ms)','Activity (spk/s)', con, connections,Mean, 'MsnFsi', 'GpeMsn','FsiGpe',On=1)
plt.savefig('DataFsi/{}/{}_Mean_Activities72.png'.format(timestr,'0'),dpi=72,bbox_inches='tight')
plt.savefig('DataFsi/{}/{}_Mean_Activities150.png'.format(timestr,'0'),dpi=150,bbox_inches='tight')
plt.savefig('DataFsi/{}/{}_Mean_Activities300.png'.format(timestr,'0'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))


ax1 = plt.subplot(311)
ax1.set_title('Mean Activities',fontsize=30)
#ax1.set_ylim(19.5,20.3)
ax1.set_ylabel('MSN',fontsize=20,fontweight='bold')
plt.plot(time_value*1000, Mean['GpeMsn'],'g',linewidth=2)
plt.setp(ax1.get_xticklabels(), visible=False)

    # share x only
ax2 = plt.subplot(312, sharex=ax1)
#ax2.set_ylim(2,6.2)
ax2.set_ylabel('FSI',fontsize=20,fontweight='bold')
plt.plot(time_value*1000, Mean['MsnFsi'],'b',linewidth=2)

# make these tick labels invisible
plt.setp(ax2.get_xticklabels(), visible=False)

# share x and y
ax3 = plt.subplot(313, sharex=ax1)
#ax3.set_ylim(19.2,20.5)
ax3.set_ylabel('GPe',fontsize=20,fontweight='bold')
ax3.set_xlabel('Time (ms)',fontsize=25,fontweight='bold')
plt.plot(time_value*1000, Mean['FsiGpe'],'r',linewidth=2)

fig.text(0.020, 0.5, "Activities (Spikes/s)", rotation="vertical", va="center",fontsize=20,fontweight='bold')
   
plt.savefig('DataFsi/{}/MeanActivity72.png'.format(timestr),dpi=72,bbox_inches='tight')
plt.savefig('DataFsi/{}/MeanActivity150.png'.format(timestr),dpi=150,bbox_inches='tight')
plt.savefig('DataFsi/{}/MeanActivity300.png'.format(timestr),dpi=300,bbox_inches='tight')
plt.show()

# ✔	 -  Fourier transform (DFT): Spectrum analysis

In [ ]:
Spectrum()

# ✔	 -  Population analysis

In [ ]:
moyFsiGpe=[]
moyGpeMsn=[]
moyMsnFsi=[]

supmoyFG=[]
supmoyGM=[]
supmoyMF=[]

for k in tqdm(np.arange(0,n,1)): 
    moyFsiGpe=[]
    moyGpeMsn=[]
    moyMsnFsi=[]
    for t in range(0,n_period):
        moyFsiGpe.append(m['FsiGpe'][t][k])
        moyGpeMsn.append(m['GpeMsn'][t][k])
        moyMsnFsi.append(m['MsnFsi'][t][k])

    supmoyFG.append(np.mean(moyFsiGpe))
    supmoyGM.append(np.mean(moyGpeMsn))
    supmoyMF.append(np.mean(moyMsnFsi))

In [ ]:
## Plotting population distribution analysis

fig, axes = plt.subplots(1, 3, figsize=(18, 5))                # create 1x3 plot + set size
fig.suptitle("Population distribution",y=1.05, fontsize=20)    # set global title


# 1st Plot ========================================================================
axes[0].hist(supmoyFG,normed=True,edgecolor='black',color ='red', hatch = '/')
axes[0].set_title("GPe",fontsize=25,fontweight='bold')
axes[0].set_xlabel("Activities",fontsize=10)
axes[0].set_ylabel('Probability density',fontsize=10)

# 2nd Plot ========================================================================
axes[1].hist(supmoyMF,normed=True,edgecolor='black',color ='cyan', hatch = '/')  
axes[1].set_title("FSI",fontsize=25,fontweight='bold')
axes[1].set_xlabel("Activities",fontsize=10)
axes[1].set_ylabel('Probability density',fontsize=10)

# 3rd Plot ========================================================================    
axes[2].hist(supmoyGM,normed=True,edgecolor='black',color ='green', hatch = '/')
axes[2].set_title("MSN",fontsize=25,fontweight='bold')
axes[2].set_xlabel("Activities",fontsize=10)
axes[2].set_ylabel('Probability density',fontsize=10)


plt.savefig('DataFsi/{}/Dsitribution.png'.format(timestr),dpi=72,bbox_inches='tight') # save fig with fixed DPi
plt.show()